In [10]:
#ali abdollahian noghabi 9913062
#maryam sadeghabadi 9913050
import random
import math
import time
import numpy as np
import copy

# simulation setting

# some game rules
settingStartingMoney = 1500
settingsSalary = 200
settingsLuxuryTax = 75
settingsPropertyTax = 200
settingJailFine = 50
settingHouseLimit = 32
settingHotelLimit = 12
settingsAllowUnEqualDevelopment = False  # default = False
settingsBankruptcyAlwaysGoesToBank = False


# players behaviour settings
behaveUnspendableCash = 0  # Money I want to will have left after buying stuff
behaveUnmortgageCoeff = 3  # 211repay mortgage if you have times this cash
behaveBuildCheapest = False
behaveBuildRandom = False


# experimental settings
# for a player named exp:
expRefuseTrade = False  # refuse to trade property
expRefuseProperty = ""  # refuse to buy this group
expHouseBuildLimit = 100  # limit houses built
expUnspendableCash = 0  # unspendable money
expBuildCheapest = False
expBuildExpensive = False
expBuildThree = False
variableStartingMoney = []# [1370, 1460, 1540, 1630] # [] to disable
bankName = "bank"

In [5]:

class Player:
    """Player class"""

    def __init__(self, name, startingMoney):
        self.name = name
        self.position = 0
        self.money = startingMoney
        self.inJail = False
        self.daysInJail = 0
        self.isBankrupt = False
        self.hasMortgages = []
        self.plotsWanted = []
        self.plotsToBuild = []
        self.cashLimit = expUnspendableCash if name == "exp" else behaveUnspendableCash

    def __str__(self):
        return "Player: "+self.name + \
               ". Position: "+str(self.position) + \
               ". Money: $"+str(self.money)

    # some getters and setters
   
    # add money (salary, receive rent etc)
    def addMoney(self, amount):
        self.money += amount

    # subtract money (pay reny, buy property etc)
    def takeMoney(self, amount, board, action_origin):
        amount_taken = min(self.money, amount)
        self.money -= amount
        final_account_balance = self.money
        self.checkBankrupcy(board, action_origin)
        if self.isBankrupt:
            amount_taken += self.money - final_account_balance
        else:
            amount_taken = amount
        return amount_taken


   # check if player has negative money
    # if so, start selling stuff and mortgage plots
    # if that's not enough, player bankrupt
    def checkBankrupcy(self, board, bankrupter):
        if self.money < 0:
            # print(self.name+" doesn't have enough cash", 3)
            self.isBankrupt = True
            if bankrupter == bankName or settingsBankruptcyAlwaysGoesToBank:
                board.sellAll(self)
                # print("The bank bankrupted " + self.name + ". Their property is back on the board", 3)
            else:
                board.sellAll(self, bankrupter)
                # print(self.name + " is now bankrupt. " + bankName + " bankrupted them", 3)
            board.recalculateAfterPropertyChange()
            # print( self.name + "lost" )
            # print("last cell" + str(self.position))
            return




    # subtract money (pay reny, buy property etc)
    def moveTo(self, position):
        self.position = position
        print(self.name+" moves to cell "+str(position))


    # make a move procedure
    def makeAMove(self, board):

        # Only proceed if player is alive (not bankrupt)
        if self.isBankrupt:
            return

        print("position: " + str(self.position))
        # print("Player "+self.name+" goes:", 2)


        if self.inJail:
          if self.daysInJail <= 3:
            self.daysInJail +=1
            return 
          else:#relase and roll dice 
            self.daysInJail = 0
            self.inJail = False

                                                                                                 
        # roll dice
        dice1 = random.randint(1, 6)
        print(self.name+" rolls "+str(dice1))


        # Jail situation:
        # move the piece
        self.position += dice1

        # correction of the position if landed on GO or overshoot GO
        if self.position >= 40:
            # calculate correct cell
            self.position = self.position - 40
            # get salary for passing GO
            self.addMoney(settingsSalary)
            print(self.name+" gets salary: $"+str(settingsSalary))

        print(self.name+" moves to cell "+str(self.position) + ": "+board.b[self.position].name +
                  (" ("+board.b[self.position].owner.name+")" if type(board.b[self.position]) == Property and board.b[self.position].owner != "" else ""))

        # perform action of the cell player ended on
        board.action(self, self.position)

        return False  # no extra move


    # Calculate net worth of a player (for property tax)
    def netWorth(self, board):
        worth = self.money
        for plot in board.b:
            if type(plot) == Property and plot.owner == self:
              if plot.isMortgaged:
                mortgage = plot.cost_base // 2
              else:
                mortgage = 0
              if plot.isMonopoly:
                b=2
              else:
                b=1.5
              worth += (plot.cost_base - mortgage)*b
              worth += plot.cost_house * plot.hasHouses // 2
        return worth
    
    # What % of plots of this group does player have
    # Used in calculation of least valuable property
    def shareOfGroup(self, group, board):
        total = 0
        owned = 0
        for plot in board.b:
            if type(plot) == Property and plot.group == group:
                total += 1
                if plot.owner == self:
                    owned += 1
        return owned / total

    # Behaviours
     
    # if there is a mortgage with pay less then current money // behaveUnmortgageCoeff
    # repay the mortgage
    def repayMortgage(self, property , board):
      if property.isMortgaged and self.money > property.unMortgageValue * behaveUnmortgageCoeff:
        property.unmortgage(self, board)
        board.recalculateAfterPropertyChange()
        return True
      return False


    # does player want to buy a property
    def wantsToBuy(self, base_cost, group, board):
        if self.money > base_cost + self.cashLimit:
          respond = input("do you want to buy this property (y/n)")
          if respond == "y":
            return True
        else:
            print(self.name + "doesn't have enough money")
            return False
    
    def getPossibleActions(self,state,roll,root):
        actions=[]
        # if self.isBankrupt:
        #     return 
        if state["currentPlayer"].inJail:
          if state["currentPlayer"].daysInJail <= 3:
            state["currentPlayer"].daysInJail +=1
            if root == True:
              print("yes")
              return 
            else:
              actions.append("skip")
              return actions,state
          else:#relase and roll dice 
            state["currentPlayer"].daysInJail = 0
            state["currentPlayer"].inJail = False
        state["currentPlayer"].position += roll
        if state["currentPlayer"].position >= 40:
          state["currentPlayer"].position = state["currentPlayer"].position - 40
          state["currentPlayer"].addMoney(settingsSalary)
        #property.action
        if type(state["currentProperty"]) == Property:#owner.self
          rent = state["board"].calculateRent(state["position"])
          if state["currentProperty"].owner == state["currentPlayer"]:
            if not state["currentProperty"].isMortgaged:
              actions.append("improve")
              actions.append("mortgage")
              actions.append("sell")
              actions.append("skip")
            elif state["currentProperty"].isMortgaged and state["currentPlayer"].money > state["currentProperty"].unMortgageValue * behaveUnmortgageCoeff:
              actions.append("repaymortgage")
              actions.append("skip")
            else:
              if root == True:
                return
              else:
                actions.append("skip")
          elif state["currentProperty"].owner == "":#owner:bank
            if state["currentPlayer"].money > state["currentProperty"].cost_base + state["currentPlayer"].cashLimit:
              actions.append("buy")
              actions.append("skip")
            else:
              if root == True:
                return
              else:
                actions.append("skip")
          elif not state["currentProperty"].isMortgaged:#owner:otherPlayer
              amount_taken = state["currentPlayer"].takeMoney(rent, state["board"], state["currentProperty"].owner)
              state["currentProperty"].owner.addMoney(amount_taken)
              if root == True:
                return
              else:
                actions.append("skip")
        #cell.action for gotojail
        elif type(state["currentProperty"]) == GoToJail:
          if state["currentPlayer"].money > settingJailFine:
            actions.append("payFine")
            actions.append("movetoJail")
          else: 
            state["currentPlayer"].position = 10
            state["currentPlayer"].inJail = True
            if root == True:
              return
            else:
              actions.append("skip")
        #cell action for tax and other cells
        else:
          state["currentProperty"].action(state["currentPlayer"],state["board"])
          if root == True:
            return
          else:
            actions.append("skip")
        return actions,state


    def getResult(self,state,action):
        if action == "improve":
          state["board"].improveProperty(state["currentProperty"].group , state["currentPlayer"] , state["currentPlayer"].money - state["currentPlayer"].cashLimit)
        elif action == "mortgage":
          state["currentProperty"].mortgage(state["currentPlayer"],state["board"])
        elif action == "sell":
          state["currentProperty"].sell(state["currentPlayer"],state["board"])
        elif action == "repaymortgage":
          state["currentProperty"].unmortgage(state["currentPlayer"], state["board"])
          state["board"].recalculateAfterPropertyChange()
        elif action == "buy" :
          state["currentPlayer"].takeMoney( state["currentProperty"].cost_base , state["board"], bankName)
          state["currentProperty"].owner = state["currentPlayer"]
          state["board"].recalculateAfterPropertyChange()
        elif action == "payFine":
          state["currentPlayer"].takeMoney(settingJailFine, state["board"], bankName)
        elif action == "movetoJail":
          state["currentPlayer"].position = 10
          state["currentPlayer"].inJail = True
        elif action == "skip":
          pass
        return state
        
    def get_reward(self,state):
        totalNetWorth = 0
        board = state["board"]
        currentPlayer = state["currentPlayer"]
        for player in board.players:
          totalNetWorth += player.netWorth(board)
        currentPlayerNetWorth = currentPlayer.netWorth(board)
        reward = currentPlayerNetWorth / totalNetWorth
        groups =["brown","rail","lightblue","pink","util","orange","red","yellow","green","indigo"]
        for group in groups:
          reward += 1.1 * currentPlayer.shareOfGroup(group, board)
        return reward


    def getOpponent(self,board):
      for i in range(len(board.players)):
        if board.players[i].name == self.name:
          return board.players[(i+1) % len(board.players)]


    def makeDecision(self,board,depth):
      state = board.getState(self)
      roll = random.randint(1, 6)
      print("AI rolls",roll)
      max_value = -np.inf
      best_action = None
      copyState = copy.deepcopy(state)
      result = self.getPossibleActions(state,roll,True)
      if result != None:
        for action in result[0]:
          value =  self.expectiNode(self.getResult(copyState,action),depth-1)
          if value>max_value:
            best_action = action
            max_value = value
        print("best_action",best_action)
        return self.getResult(state,best_action)


    def expectiNode(self,state,depth):
      if depth == 0:
        return self.get_reward(state)
        # Otherwise, compute the expected value of the next state
      else:
        outcome_values = []
        state["currentPlayer"] = self.getOpponent(state["board"])
        for roll in range(1,7):
          if state["currentPlayer"].name == "AI":
            outcome_value = self.maxValue(state,roll,depth-1)
          else:
            outcome_value = self.minValue(state,roll,depth-1)
          outcome_values.append(outcome_value)
        expected_outcome_value = np.mean(outcome_values)
        return expected_outcome_value


    def maxValue(self,state,roll,depth):
      actions,currentState = self.getPossibleActions(state,roll,False) 
      if  depth == 0:
            return self.get_reward(currentState)
      else:
        max_value = -np.inf
        for action in actions:
          expected_outcome_value = max(max_value, self.expectiNode(self.getResult(state,action),depth-1))
      return max_value


    def minValue(self,state,roll,depth):
      actions,currentState = self.getPossibleActions(state,roll,False)
      if  depth == 0:
            return self.get_reward(currentState)
      else:
        min_value = np.inf
        for action in actions:
          min_value = min(min_value, self.expectiNode(self.getResult(state,action),depth-1))
      return min_value



In [6]:
class Cell:
    """Generic Cell Class, base for other classes"""

    def __init__(self, name):
        self.name = name
        self.group = ""

    def action(self, player, board):
        pass


class LuxuryTax(Cell):
    """Pay Luxury Tax cell (#38)"""
    def action(self, player, board):
        player.takeMoney(settingsLuxuryTax, board, bankName)
        print(player.name+" pays Luxury Tax $"+str(settingsLuxuryTax))


class PropertyTax(Cell):
    """Pay Property Tax cell (200 or 10%) (#4)"""
    def action(self, player, board):
        print(player.name+" pays Property Tax $")
        player.takeMoney(settingsPropertyTax, board, bankName)


class GoToJail(Cell): 
    """Go to Jail (#30)"""
    def action(self, player, board):
        if player.money > settingJailFine:
          respond = input("do you want to pay 50 dollars(p) or go to jail(j) (p/j")
          if respond == "p": #player chooses to pay fine
            player.takeMoney(settingJailFine, board, bankName)
          else: #player chooses to go to jail
            player.position = 10
            player.inJail = True
            print(player.name+" goes to jail from Go To Jail ")
        else: #player doesn't have enough money to pay the fine
          player.position = 10
          player.inJail = True
          print(player.name+" goes to jail from Go To Jail ")




In [7]:

class Property(Cell):
    """Property Class (for Properties, Rails, Utilities)"""

    def __init__(self, name, cost_base, rent_base, cost_house, rent_house, group):
        self.name = name
        self.cost_base = cost_base
        self.rent_base = rent_base
        self.cost_house = cost_house
        self.rent_house = rent_house
        self.group = group
        self.owner = ""
        self.isMortgaged = False
        self.unMortgageValue = int((self.cost_base // 2) * 1.1)
        self.isMonopoly = False
        self.hasHouses = 0


    def action(self, player, rent, board):
        """Player ended on a property"""

        if self.owner == player:
          respond = input("do you want to upgrade or unmortgage or do nothing (1/2/3)")
          if respond == "1" and not self.isMortgaged:
            board.improveProperty(self.group , player, player.money - player.cashLimit)
          elif respond == "2":
            player.repayMortgage(self , board)
          elif respond == "3" and not self.isMortgaged:
            self.mortgage(player,board)
          elif respond =="4" and not self.isMortgaged:
            self.sell(player,board)
          else:
            print("you refused to upgrade or unmortgage")

        # Property up for sale
        elif self.owner == "":
            if player.wantsToBuy(self.cost_base ,  self.group, board):
                print(player.name+" buys property " + self.name + " for $"+str(self.cost_base))
                player.takeMoney(self.cost_base, board, bankName)
                self.owner = player
                board.recalculateAfterPropertyChange()
            return

        # someone else's property - pay the rent if it's not mortgaged
        elif not self.isMortgaged:
            amount_taken = player.takeMoney(rent, board, self.owner)
            self.owner.addMoney(amount_taken)
            print(player.name+" pays the rent $" + str(rent) + " to "+self.owner.name)


    # or sell the house
    def sell(self, player, board):
        """Sell hotel"""
        if self.hasHouses == 5:
            player.addMoney(self.cost_house * 5 // 2)
            self.hasHouses = 0
            board.nHotels -= 1
            print(player.name+" sells hotel on "+self.name)
            board.recalculateAfterPropertyChange()
        # Sell one house
        elif self.hasHouses > 0:
            player.addMoney(self.cost_house // 2)
            self.hasHouses -= 1
            board.nHouses -= 1
            print(player.name+" sells house on "+self.name)
            board.recalculateAfterPropertyChange()
        else:
           print(self.name+" doesn't have any house ")

      # Mortgage
    def mortgage(self, player, board):
        self.isMortgaged = True
        player.addMoney(self.cost_base // 2)
        # log name of the plot and money player need to pay to get it back
        # player.hasMortgages.append((self, int((self.cost_base // 2) * 1.1)))
        print(player.name+" mortgages "+self.name)
        board.recalculateAfterPropertyChange()


    # unmortgage thr plot
    def unmortgage(self, player, board):
      # for mortgage in player.hasMortgages:
      #   if mortgage[0] == self:
      #     thisMortgage = mortgage
      self.isMortgaged = False
      player.takeMoney(self.unMortgageValue , board, bankName)
      player.hasMortgages.remove(self)
      print(player.name+" unmortgages "+self.name)


In [8]:
class Board:

    def __init__(self, players):
        """
        Board is a data for plots
        name: does not really matter, just convenience
        base_cost: used when buying plot, mortgage
        base_rent: used for rent and monopoly rent (for utilities and rail - in calculateRent method)
        house_cost: price of one house (or a hotel)
        house_rent: list of rent price with 1,2,3,4 houses and a hotel
        group: used to determine monopoly
        """

        # I know it is messy, but I need this for players to pay each other
        self.players = players
        self.b = []
        # 0-4
        self.b.append(Cell("Go"))
        self.b.append(Property("A1 Mediterraneal Avenue", 60, 2, 50, (10, 30, 90, 160, 250), "brown"))
        self.b.append(Cell("Free Parking" ))
        self.b.append(Property("A2 Baltic Avenue", 60, 4, 50, (20, 60, 180, 320, 450), "brown"))
        self.b.append(PropertyTax("Property Tax"))
        # 5-9
        self.b.append(Property("R1 Reading railroad", 200, 0, 0, (0, 0, 0, 0, 0), "rail"))
        self.b.append(Property("B1 Oriental Avenue", 100, 6, 50, (30, 90, 270, 400, 550), "lightblue"))
        self.b.append(Cell("Free Parking" ))
        self.b.append(Property("B2 Vermont Avenue", 100, 6, 50, (30, 90, 270, 400, 550), "lightblue"))
        self.b.append(Property("B3 Connecticut Avenue", 120, 8, 50, (40, 100, 300, 450, 600), "lightblue"))
        # 10-14
        self.b.append(Cell("Prison"))
        self.b.append(Property("C1 St.Charle's Place", 140, 10,100, (50, 150, 450, 625, 750), "pink"))
        self.b.append(Property("U1 Electric Company",  150, 0, 0, (0, 0, 0, 0, 0), "util"))
        self.b.append(Property("C2 States Avenue", 140, 10, 100, (50, 150, 450, 625, 750), "pink"))
        self.b.append(Property("C3 Virginia Avenue", 160, 12, 100, (60, 180, 500, 700, 900), "pink"))
        # 15-19
        self.b.append(Property("R2 Pennsylvania Railroad",200, 0, 0, (0, 0, 0, 0, 0), "rail"))
        self.b.append(Property("D1 St.James Place", 180, 14, 100, (70, 200, 550, 700, 950), "orange"))
        self.b.append(Cell("Free Parking" ))
        self.b.append(Property("D2 Tennessee Avenue", 180, 14, 100, (70, 200, 550, 700, 950), "orange"))
        self.b.append(Property("D3 New York Avenue", 200, 16, 100, (80, 220, 600, 800, 1000), "orange"))
        # 20-24
        self.b.append(Cell("Free Parking" ))
        self.b.append(Property("E1 Kentucky Avenue", 220, 18, 150, (90, 250, 700, 875, 1050), "red"))
        self.b.append(Cell("Free Parking" ))
        self.b.append(Property("E2 Indiana Avenue", 220, 18, 150, (90, 250, 700, 875, 1050), "red"))
        self.b.append(Property("E3 Illinois Avenue", 240, 20, 150, (100, 300, 750, 925, 1100), "red"))
        # 25-29
        self.b.append(Property("R3 BnO Railroad", 200, 0, 0, (0, 0, 0, 0, 0), "rail"))
        self.b.append(Property("F1 Atlantic Avenue", 260, 22, 150, (110, 330, 800, 975, 1150), "yellow"))
        self.b.append(Property("F2 Ventinor Avenue", 260, 22, 150, (110, 330, 800, 975, 1150), "yellow" ))
        self.b.append(Property("U2 Waterworks", 150, 0, 0, (0, 0, 0, 0, 0), "util" ))
        self.b.append(Property("F3 Martin Gardens", 280, 24, 150, (120, 360, 850, 1025, 1200), "yellow" ))
        # 30-34
        self.b.append(GoToJail("Go To Jail"))
        self.b.append(Property("G1 Pacific Avenue", 300, 26, 200, (130, 390, 900, 1100, 1275), "green"))
        self.b.append(Property("G2 North Carolina Avenue", 300, 26, 200, (130, 390, 900, 1100, 1275), "green"))
        self.b.append(Cell("Free Parking" ))
        self.b.append(Property("G3 Pennsylvania Avenue", 320, 28, 200, (150, 450, 100, 1200, 1400), "green"))
        # 35-39
        self.b.append(Property("R4 Short Line", 200, 0, 0, (0, 0, 0, 0, 0), "rail"))
        self.b.append(Cell("Free Parking" ))
        self.b.append(Property("H1 Park Place", 350, 35, 200, (175, 500, 1100, 1300, 1500), "indigo"))
        self.b.append(LuxuryTax("Luxury Tax"))
        self.b.append(Property("H2 Boardwalk", 400, 50, 200, (200, 600, 1400, 1700, 2000), "indigo" ))

        # number of built houses and hotels (to limit when needed)
        self.nHouses = 0
        self.nHotels = 0
    

    def getState(self,currentPlayer):
      state={"currentPlayer":currentPlayer,
             "board":self,
             "position":currentPlayer.position,
             "players":self.players,
             "currentProperty": self.b[currentPlayer.position]}
      return state

    # Does the board have at least one monopoly
    # Used for statistics
    def hasMonopoly(self):
        for i in range(len(self.b)):
            if self.b[i].isMonopoly:
                return True
        return False

    # Count the number of rails of the same owner as "position"
    # Used in rent calculations
    def countRails(self, position):
        railcount = 0
        thisOwner = self.b[position].owner
        for plot in self.b:
            if type(plot) == Property and plot.group == "rail" \
               and plot.owner == thisOwner and plot.owner != "" \
               and not plot.isMortgaged:
                railcount += 1
        return railcount


    # What is the rent of plot "position"
    # Takes into account utilities, rails, monopoly
    def calculateRent(self, position, special=""):
        if type(self.b[position]) == Property:
            rent = 0

            # utility
            if self.b[position].group == "util":
                if self.b[position].isMonopoly :
                    rent = random.randint(1, 6) * 10
                else:
                    rent = random.randint(1, 6) * 4

            # rail
            elif self.b[position].group == "rail":
                rails = self.countRails(position)
                rent = 0 if rails == 0 else 25 * 2**(rails)

            # usual property
            else:
                if self.b[position].hasHouses > 0:
                    rent = self.b[position].rent_house[self.b[position].hasHouses-1]
                elif self.b[position].isMonopoly:
                    rent = 2*self.b[position].rent_base
                else:
                    rent = self.b[position].rent_base
        else:  # not a Property
            rent = 0
        return rent



    # Chose Property to build the next house/hotel according to its value and available money
    def listPropertyToBuild(self, player):
        # list all the items this player could built on:
        toBuildStuff = []
        # smaller level of improvement in the group (to prevent inequal improvement)
        minInGroup = {}
        # start with listing all their monopolies
        for i in range(len(self.b)):
            plot = self.b[i]
            if type(plot) == Property and self.b[i].isMonopoly and plot.owner == player \
               and plot.group != "rail" and plot.group != "util" and plot.hasHouses < 5:
                # limit max houses experiment
                if not (player.name == "exp" and expHouseBuildLimit == plot.hasHouses):
                    toBuildStuff.append((i, plot.name, plot.group, plot.hasHouses, plot.cost_house, plot.cost_base))
                    if plot.group in minInGroup:
                        minInGroup[plot.group] = min(plot.hasHouses, minInGroup[plot.group])
                    else:
                        minInGroup[plot.group] = plot.hasHouses
        if len(toBuildStuff) == 0:
            return []
        # remove those that has more houses than other plots in monopoly (to ensure gradual development)
        toBuildStuff.sort(key=lambda x: (x[2], x[3]))
        for i in range(len(toBuildStuff)-1, -1, -1):
            # if it has more houses than minimum in that group, remove
            if toBuildStuff[i][3] > minInGroup[toBuildStuff[i][2]]:
                if not settingsAllowUnEqualDevelopment:#False
                    toBuildStuff.pop(i)
        toBuildStuff.sort(key=lambda x: (x[4], x[5]))#Done
        return toBuildStuff


    def choosePropertyToBuild(self, propertyGroup , player, availiableMoney):
        for i in range(len(player.plotsToBuild)-1, -1, -1):
            if player.plotsToBuild[i][2] == propertyGroup and player.plotsToBuild[i][4] <= availiableMoney:
                return player.plotsToBuild[i][0]
        return False


    # Build one house/hotel withinavailabel money
    # return True if built, so this function will be called again
    def improveProperty(self, propertyGroup,player , availiableMoney):
        propertyToImprove = self.choosePropertyToBuild(propertyGroup , player, availiableMoney)
        if type(propertyToImprove) == bool and not propertyToImprove:
            return False

        # Check if we reached the limit of available Houses/Hotels
        thisIsHotel = True if self.b[propertyToImprove].hasHouses == 4 else False
        if thisIsHotel:
            if self.nHotels == settingHotelLimit:
                print("reached hotel limit")
                return False
        else:
            if self.nHouses == settingHouseLimit:
                print("reached house limit")
                return False

        # add a building
        self.b[propertyToImprove].hasHouses += 1
        # add to the counter
        if thisIsHotel:
            self.nHotels += 1
            self.nHouses -= 4
        else:
            self.nHouses += 1

        print(player.name+" builds house N" + str(self.b[propertyToImprove].hasHouses)+" on "+self.b[propertyToImprove].name)
        player.takeMoney(self.b[propertyToImprove].cost_house, self, bankName)
        player.plotsToBuild = self.listPropertyToBuild(player)
        return True


    # When player is bankrupt - return all their property to market
    def sellAll(self, player, other_player=None):
        for plot in self.b:
            if type(plot) == Property and plot.owner == player:
                if other_player is None:
                    plot.owner = ""
                    plot.isMortgaged = False
                else:
                    plot.owner = other_player


    # Get the list of plots player would want to get
    # that is he lacks one to for a monopoly
    def getListOfWantedPlots(self, player):
        groups = {}
        for plot in self.b:
            if plot.group != "":
                if plot.group in groups:
                    groups[plot.group][0] += 1
                else:
                    groups[plot.group] = [1, 0]
                if plot.owner == player:
                    groups[plot.group][1] += 1
        wanted = []
        for group in groups:
            if group != "util" and groups[group][0]-groups[group][1] == 1:
                for i in range(len(self.b)):
                    if type(self.b[i]) == Property and self.b[i].group == group and self.b[i].owner != player:
                        wanted.append(i)
        return sorted(wanted)


    # update isMonopoly status for all plots
    def checkMonopolies(self):
        groups = {}
        for i in range(len(self.b)):
            plot = self.b[i]
            if type(plot) == Property:
                if plot.owner == "":
                    groups[plot.group] = False
                else:
                    if plot.group in groups:
                        if groups[plot.group] != plot.owner:
                           groups[plot.group] = False
                    else:
                        groups[plot.group] = plot.owner
        for i in range(len(self.b)):
            plot = self.b[i]
            if type(plot) == Property:
                if groups[plot.group] != False:
                    plot.isMonopoly = True
                else:
                    plot.isMonopoly = False


    # calculating heavy tasks that we want to do after property change:
    # list of wanted and offered properties for each player
    def recalculateAfterPropertyChange(self):
        self.checkMonopolies()
        for player in self.players:
            player.plotsWanted = self.getListOfWantedPlots(player)
            player.plotsToBuild = self.listPropertyToBuild(player)


    # perform action for a player on a plot
    def action(self, player, position, special=""):
        # Landed on a property - calculate rent first
        if type(self.b[position]) == Property:
            # calculate the rent one would have to pay (but not pay it yet)
            rent = self.calculateRent(position, special=special)
            # pass action to to the property
            self.b[position].action(player, rent, self)
        else:
            # pass action to to the cell
            self.b[position].action(player, self)


In [9]:

def isGameOver(players):
    """Check if there are more then 1 player left in the game"""
    alive = 0
    for player in players:
        if not player.isBankrupt:
            alive += 1
    if alive == 1:
        return True
    else:
        return False


if __name__ == "__main__":
    # create players
    players = []
    players.append(Player("Human", settingStartingMoney))#start by human
    players.append(Player("AI", settingStartingMoney))
 

    # create board
    gameBoard = Board(players)
    
    i=0
    while not isGameOver(players): # Only continue if 2 players
        i = i % len(players)
       # returns True if player has to go again
        print(f"\n{players[i].name}'s turn:")
        print(f"{players[0].name}'s money: {players[0].money}")
        print(f"{players[1].name}'s money: {players[1].money}")

        if players[i].name == "AI":
           best_action=players[i].makeDecision(gameBoard,depth=6)
        else:
          players[i].makeAMove(gameBoard)

        i += 1

    if(players[0].isBankrupt):
      print(players[1].name , "wins!")
    else:
      print(players[0].name , "wins!")


Human's turn:
Human's money: 1500
AI's money: 1500
position: 0
Human rolls 6
Human moves to cell 6: B1 Oriental Avenue
do you want to buy this property (y/n)y
Human buys property B1 Oriental Avenue for $100

AI's turn:
Human's money: 1400
AI's money: 1500
AI rolls 1

Human's turn:
Human's money: 1400
AI's money: 1500
position: 6
Human rolls 3
Human moves to cell 9: B3 Connecticut Avenue
do you want to buy this property (y/n)n

AI's turn:
Human's money: 1400
AI's money: 1500
AI rolls 6
best_action skip

Human's turn:
Human's money: 1400
AI's money: 1500
position: 9
Human rolls 5
Human moves to cell 14: C3 Virginia Avenue
do you want to buy this property (y/n)y
Human buys property C3 Virginia Avenue for $160

AI's turn:
Human's money: 1240
AI's money: 1500
AI rolls 6

Human's turn:
Human's money: 1240
AI's money: 1500
position: 14
Human rolls 6
Human moves to cell 20: Free Parking

AI's turn:
Human's money: 1240
AI's money: 1500
AI rolls 4
best_action buy

Human's turn:
Human's money: 1